In [ ]:
import pandas as pd

## Load the data

In [ ]:
data = pd.read_csv("/kaggle/input/paris-housing-classification/ParisHousingClass.csv") 

## Check the data

In [ ]:
data.head()

In [ ]:
data.size

In [ ]:
data.info()


In [ ]:
data.describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder

data["category"] = LabelEncoder().fit_transform(data["category"])
data

In [ ]:
prepared_data = data.copy()

In [ ]:
prepared_data.info()

In [ ]:
print(prepared_data.isnull().sum())

In [ ]:
housing_category_labels = prepared_data["category"].copy()
prepared_data = prepared_data.drop(["category"], axis=1)
prepared_data

## 1. Separe el dataset en Train y Test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(prepared_data, housing_category_labels, test_size=0.30, random_state=42)

## 2. Realice Análisis de los datos (EDA) con gráficos o tablas

In [ ]:
import matplotlib.pyplot as plt
data.hist(bins=6, figsize=(20,15))
plt.show()

## 3. Grafique la matriz de correlación

In [ ]:
import plotly.express as px

fig = px.imshow(data.corr())

fig.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

need_scaling = ['squareMeters', 'numberOfRooms', 'floors', 'cityCode', 'made', 'basement', 'attic', 'garage', 'price']

standard_scaler = StandardScaler()
prepared_data[need_scaling] = standard_scaler.fit_transform(prepared_data[need_scaling])

## 4. Ajuste un modelo utilizando regresión logística

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg_model = LogisticRegression(random_state=0, fit_intercept=False).fit(X_train, y_train)

## 5. Muestre el accuracy del modelo utilizando tanto el train y como el test.

In [ ]:
accuracy = logreg_model.score(X_train, y_train)
test_accuracy = logreg_model.score(X_test, y_test)
print(f"accuracy: {accuracy}, test_accuracy: {test_accuracy}")

### Regularización L1 (Lasso)

In [ ]:
c_vals = [100, 10, 0.05, 0.001]
for c in c_vals:
    logreg_l1_model = LogisticRegression(C=c, penalty="l1", solver="liblinear", max_iter=2000)
    logreg_l1_model.fit(X_train, y_train)
    accuracy = logreg_l1_model.score(X_train, y_train)
    test_accuracy = logreg_l1_model.score(X_test, y_test)
    print(f"c: {c}, accuracy: {accuracy}, test_accuracy: {test_accuracy}")

### Regularización L2 (Ridge)

In [ ]:
c_vals = [100, 10, 0.05, 0.001]
for c in c_vals:
    logreg_l2_model = LogisticRegression(C=c, penalty="l2", solver="lbfgs")
    logreg_l2_model.fit(X_train, y_train)
    accuracy = logreg_l2_model.score(X_train, y_train)
    test_accuracy = logreg_l2_model.score(X_test, y_test)
    print(f"c: {c}, accuracy: {accuracy}, test_accuracy: {test_accuracy}")

## Relevancia

In [ ]:
logreg_model.coef_[0]

## 6. Cuales son las 2 características (features) mas relevantes a la hora de realizar la clasificación

In [ ]:
import numpy as np
features_relevancy = pd.DataFrame({
    'feature': X_test.columns,
    'coef': np.abs(logreg_model.coef_[0])
})

In [ ]:
features_relevancy.sort_values(by='coef', ascending=True)

Las mas relevantes son made y garage

## 7. Cuales son las 2 características (features) menos relevantes a la hora de realizar la clasificación.

las menos relevantes son hasStorageRoom, hasStormProtector

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_train, logreg_model.predict(X_train))

In [ ]:
log_loss(y_test, logreg_model.predict(X_test))

In [ ]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()